In this notebook, we measure the sequence similarity between query sequences and PDB sequences using Smith-Waterman distance

In [2]:
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from collections import Counter

### Data

In [2]:
%store -r df_10k

In [4]:
df_10k['Lenght_difference']= df_10k['pdb_sequence_length'] - df_10k['query_sequence_length']

In [5]:
%store df_10k

Stored 'df_10k' (DataFrame)


In [6]:
df_10k_5=df_10k[(df_10k["Lenght_difference"]>=-5) & (df_10k["Lenght_difference"]<=5)].copy()

In [11]:
#!{sys.executable} -m pip install textdistance
import textdistance

In [9]:
def get_smith_waterman_distance(data):
    """ Smith-Waterman performs a local alignment to find the best alignment over the conserved domain
    of two sequences. It is useful for dissimilar sequences that are suspected to contain regions of 
    similarity or similar sequence motifs within their larger sequence context. The Smith Waterman algorithm is 
    used to perform local alignments of sequence; it is helpful in determining identical regions between two protein 
    sequences. This algorithm compares segments of all possible lengths and optimizes similarity. This algorithm uses
    dynamic programming to find the optimal local alignment with respect to scoring system used. Compared to 
    Needleman-Wunsch Algorithm, this algorithm sets the negative scoring matrix cells to zero, thus positive score 
    are only visible for backtracking, which is done by starting with highest scoring matrix cell and proceeds until 
    a cell with zero is encountered, which gives the highest scoring local alignment. """
    
    SW_list = []
    
    for i in range(0, len(data)):
        
            SW_list_i =  round((textdistance.smith_waterman.normalized_similarity(str(data.query_sequences.iloc[i]), str(data.pdb_sequence.iloc[i])))*100, 1)
            SW_list.append(SW_list_i)
    
    return SW_list          

## All results

In [12]:
dist_all = get_smith_waterman_distance(df_10k) 
df_10k['Smith_Waterman_distance'] = dist_all
df_10k.to_csv('./grampa_pdb_10k.csv')

## -5 to 5 length difference

In [13]:
dist_5 = get_smith_waterman_distance(df_10k_5)
df_10k_5['Smith_Waterman_distance'] = dist_5


In [14]:
df_10k_5

,query_sequences,query_sequence_length,pdb_sequence,pdb_sequence_length,pdb_id,e_value,percentage_identity,molecule_name,Lenght_difference,Smith_Waterman_distance
0,GLPRKILCAIAKKKGKCKGPLKLVCKC,27.0,GLPRKILCAIAKKKGKCKGPLKLVCKC,27.0,2mbd,1.100000e-13,100.0,[Lasiocepsin],0.0,100.0
1,GFGCPGDAYQCSEHCRALGGGRTGGYCAGPWYLGHPTCTCSF,42.0,GFGCPGDAYQCSEHCRALGGGRTGGYCAGPWYLGHPTCTCSF,42.0,2lt8,7.800000e-22,100.0,[Fungal defensin eurocin],0.0,100.0
2,GFGCPGDAYQCSEHCRALGGGRTGGYCAGPWYLGHPTCTCSF,42.0,GFGCPNNYQCHRHCKSIPGRCGGYCGGWHRLRCTCYRCG,39.0,1fjn,4.500000e-06,52.5,[Defensin MGD-1],-3.0,43.6
3,GFGCPGDAYQCSEHCRALGGGRTGGYCAGPWYLGHPTCTCSF,42.0,GFGCPFNENECHAHCLSIGRKFGFCAGPLRATCTCGKQ,38.0,2lr5,7.400000e-05,50.0,[micasin],-4.0,39.5
5,GFGCPGDAYQCSEHCRALGGGRTGGYCAGPWYLGHPTCTCSF,42.0,GFGCPFNQGKCHRHCRSIRRRGGYCDGFLKQRCVCYRK,38.0,5xa6,8.500000e-04,45.0,[INVERT_DEFENSINS domain-containing protein],-4.0,31.6
...,...,...,...,...,...,...,...,...,...,...
66800,KLWKLFKKIGIGAVLKVLTTGLPALKLTK,29.0,GIGAVLKVLTTGLPALISWIKRKRQQX,27.0,2mlt,1.000000e-03,100.0,[Melittin],-2.0,37.0
66801,KLWKLFKKIGIGAVLKVLTTGLPALKLTK,29.0,GIGAVLKVLTTGLPALISWIKRKRQQX,27.0,6o4m,1.000000e-03,100.0,[Melittin],-2.0,37.0
66802,KLWKLFKKIGIGAVLKVLTTGLPALKLTK,29.0,GIGAVLKVLTTGLPALISWIKRKRQQX,27.0,1bh1,1.000000e-03,100.0,[Melittin],-2.0,37.0
66803,KLWKLFKKIGIGAVLKVLTTGLPALKLTK,29.0,GIGKFLKKAKKGIGAVLKVLTTGL,24.0,2k98,4.800000e-02,76.2,[MSI-594],-5.0,37.5


In [14]:
df_10k_5.to_csv('./grampa_pdb_5_SWD.csv')

NameError: name 'df_10k_5' is not defined

# EDA

In [46]:
df_10k_5.molecule_name = df_10k_5.molecule_name.astype(str)

In [48]:
len(Counter(df_10k_5.molecule_name))

539

In [33]:
len(Counter(sub_res_5['pdb_sequence']))

495

In [4]:
df_10k_5=pd.read_csv('grampa_pdb_5_SWD_10k.csv', index_col=0)

In [5]:
df_10k_filt = df_10k_5[df_10k_5.Smith_Waterman_distance>=70]

In [6]:
df_10k_filt50=df_10k_filt[df_10k_filt['pdb_sequence_length']<=50]

In [7]:
len(Counter(df_10k_filt50.pdb_id))

435

In [8]:
id_list=df_10k_filt50.pdb_id.to_list()

In [7]:
%store id_list

Stored 'id_list' (list)


In [9]:
len(Counter(df_10k_filt50.pdb_sequence))

320

In [10]:
# Separamos secuencias encontradas en la busqueda inicial (10 resultados) para comparar con la busqueda de 10k 

sec_file = open('./Data/sec_PSSP.fasta', 'r')
sec_search_10r = []
for line in sec_file:
    sec_search_10r.append(line.strip())
sec_file.close()

In [11]:
pep2d_df=pd.read_csv("./Data/PEP2D.csv")

In [12]:
sec_pep2d=pep2d_df.peptide_sequence.tolist()

In [13]:
sec_df1ok = df_10k_filt50.pdb_sequence.tolist()
sec_df1ok_uniq=[]
[sec_df1ok_uniq.append(x) for x in sec_df1ok if x not in sec_df1ok_uniq]
len(sec_df1ok_uniq)

320

In [14]:
rem_list=[]
[rem_list.append(x) for x in sec_df1ok_uniq if x not in sec_search_10r]
len(rem_list)

37

In [15]:
print(rem_list[0])

KCNTATCATQRLANFLVRSSNNLGPVLPPTNVGSNTYX


In [16]:
one_id=[]
one_id_sec=[]
mult_id=[]
mult_id_sec=[]
for i in range(len(sec_df1ok_uniq)):
    testid=df_10k_filt50[df_10k_filt50.pdb_sequence == sec_df1ok_uniq[i]]['pdb_id'].to_string(index=False)
    if len(Counter(df_10k_filt50[df_10k_filt50['pdb_sequence']==sec_df1ok_uniq[i]]['pdb_id'])) == 1:
        one_id.append(testid)
        one_id_sec.append(sec_df1ok_uniq[i])
    else:
        mult_id.append(testid)
        mult_id_sec.append(sec_df1ok_uniq[i])

In [65]:
link = './Data/one_id.fasta'
ofile = open(link, "w")

for i in range(len(one_id)):
    ofile.write(">" + one_id[i] + "\n" + one_id_sec[i] + "\n")
    
ofile.close()

In [66]:
link2 = './Data/mult_id.fasta'
ofile2 = open(link2, "w")

for i in range(len(mult_id)):
    ofile2.write(">" + mult_id[i] + "\n" + mult_id_sec[i] + "\n")
    
ofile2.close()

In [17]:
for i in range(len(one_id)):
    link = './Data/psipred/' + one_id[i] + '.fasta'
    ofile = open(link, "w")
    ofile.write(">" + one_id[i] + "\n" + one_id_sec[i] + "\n")
    ofile.close()

OSError: [Errno 22] Invalid argument: './Data/psipred/ 2mbd\n 2mbd\n 2mbd\n 2mbd.fasta'

In [85]:
Counter(df_10k_filt50[df_10k_filt50['pdb_sequence']=='SGSDGGVCPKILKKCRRDSDCPGACICRGNGYCG']['pdb_id'])

Counter({'1ha9': 2, '1ib9': 2})

In [86]:
len(mult_id_sec)

59

In [87]:
len(one_id_sec)

261